<a href="https://colab.research.google.com/github/Oluhle080/MUT-Shuttle-Graph/blob/main/22325286_O_Luthuli.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import zipfile

with zipfile.ZipFile("archive (1).zip", 'r') as zip_ref:
    zip_ref.extractall()



In [ ]:
import pandas as pd

patients = pd.read_csv("patients.csv")
appointments = pd.read_csv("appointments.csv")
billing = pd.read_csv("billing.csv")
treatments = pd.read_csv("treatments.csv")
doctors = pd.read_csv("doctors.csv")



In [ ]:
# Check the first few rows of each file to see what data they contain
print(patients.head())
print(appointments.head())
print(billing.head())
print(treatments.head())
print(doctors.head())

# Check columns in each dataframe to understand keys and features
print(patients.columns)
print(appointments.columns)
print(billing.columns)
print(treatments.columns)
print(doctors.columns)


  patient_id first_name last_name gender date_of_birth  contact_number  \
0       P001      David  Williams      F    1955-06-04      6939585183   
1       P002      Emily     Smith      F    1984-10-12      8228188767   
2       P003      Laura     Jones      M    1977-08-21      8397029847   
3       P004    Michael   Johnson      F    1981-02-20      9019443432   
4       P005      David    Wilson      M    1960-06-23      7734463155   

        address registration_date insurance_provider insurance_number  \
0   789 Pine Rd        2022-06-23       WellnessCorp        INS840674   
1  321 Maple Dr        2022-01-15        PulseSecure        INS354079   
2  321 Maple Dr        2022-02-07        PulseSecure        INS650929   
3    123 Elm St        2021-03-02        HealthIndia        INS789944   
4    123 Elm St        2021-09-29       MedCare Plus        INS788105   

                      email  
0   david.williams@mail.com  
1      emily.smith@mail.com  
2      laura.jones@mail.co

Table Relationships

appointments is linked to patients using patient_id

appointments is linked to doctors using doctor_id

treatments is linked to appointments using appointment_id

billing is linked to treatments using treatment_id

billing is also linked to patients using patient_id



In [ ]:

# Merge appointments with patients
df = appointments.merge(patients, on='patient_id', how='left')

# Merge with doctors
df = df.merge(doctors, on='doctor_id', how='left')

# Merge with treatments
df = df.merge(treatments, on='appointment_id', how='left')

# Merge with billing
df = df.merge(billing, on='treatment_id', how='left')

# Show the first few rows of the merged dataset
df.head()


,appointment_id,patient_id_x,doctor_id,appointment_date,appointment_time,reason_for_visit,status,first_name_x,last_name_x,gender,...,treatment_type,description,cost,treatment_date,bill_id,patient_id_y,bill_date,amount,payment_method,payment_status
0,A001,P034,D009,2023-08-09,15:15:00,Therapy,Scheduled,Alex,Smith,F,...,Chemotherapy,Basic screening,3941.97,2023-08-09,B001,P034,2023-08-09,3941.97,Insurance,Pending
1,A002,P032,D004,2023-06-09,14:30:00,Therapy,No-show,Alex,Moore,M,...,MRI,Advanced protocol,4158.44,2023-06-09,B002,P032,2023-06-09,4158.44,Insurance,Paid
2,A003,P048,D004,2023-06-28,8:00:00,Consultation,Cancelled,Emily,Miller,M,...,MRI,Standard procedure,3731.55,2023-06-28,B003,P048,2023-06-28,3731.55,Insurance,Paid
3,A004,P025,D006,2023-09-01,9:15:00,Consultation,Cancelled,Robert,Wilson,M,...,MRI,Basic screening,4799.86,2023-09-01,B004,P025,2023-09-01,4799.86,Insurance,Failed
4,A005,P040,D003,2023-07-06,12:45:00,Emergency,No-show,Emily,Williams,M,...,ECG,Standard procedure,582.05,2023-07-06,B005,P040,2023-07-06,582.05,Credit Card,Pending


In [ ]:
# Choose the target column
# We'll predict if a patient shows up to their appointment (status)
target = 'status'
df = df.dropna(subset=[target])  # Drop rows with missing status

# Create y (target) and X (features)
y = df[target]

# Drop identifiers and columns we don't want as features
X = df.drop(['status', 'payment_status', 'appointment_id', 'bill_id', 'treatment_id', 'patient_id_x', 'patient_id_y', 'doctor_id'], axis=1)

KeyError: "['patient_id'] not found in axis"

In [ ]:
df.columns


Index(['appointment_id', 'patient_id_x', 'doctor_id', 'appointment_date',
       'appointment_time', 'reason_for_visit', 'status', 'first_name_x',
       'last_name_x', 'gender', 'date_of_birth', 'contact_number', 'address',
       'registration_date', 'insurance_provider', 'insurance_number',
       'email_x', 'first_name_y', 'last_name_y', 'specialization',
       'phone_number', 'years_experience', 'hospital_branch', 'email_y',
       'treatment_id', 'treatment_type', 'description', 'cost',
       'treatment_date', 'bill_id', 'patient_id_y', 'bill_date', 'amount',
       'payment_method', 'payment_status'],
      dtype='object')

In [ ]:
# Set the target
target = 'status'
df = df.dropna(subset=[target])  # Drop rows with missing target

# Create y (what we want to predict)
y = df[target]

# Create X (the features) — drop target and ID columns
X = df.drop([
    'status',                # target
    'payment_status',        # extra outcome column
    'appointment_id',
    'bill_id',
    'treatment_id',
    'patient_id_x',
    'patient_id_y',
    'doctor_id'
], axis=1, errors='ignore')  # don't crash if column missing


In [ ]:
# Turn text categories into numbers
X = pd.get_dummies(X, drop_first=True)


In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=42
)


In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

# Create and train Logistic Regression model
log_reg = LogisticRegression(max_iter=10000)
log_reg.fit(X_train, y_train)

# Predict on test set
y_pred_lr = log_reg.predict(X_test)

# Evaluate Logistic Regression
print("Logistic Regression Accuracy:", accuracy_score(y_test, y_pred_lr))
print("\nClassification Report:\n", classification_report(y_test, y_pred_lr))
print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred_lr))


Logistic Regression Accuracy: 0.23333333333333334

Classification Report:
               precision    recall  f1-score   support

   Cancelled       0.27      0.40      0.32        15
   Completed       0.00      0.00      0.00        18
     No-show       0.19      0.43      0.27        14
   Scheduled       0.29      0.15      0.20        13

    accuracy                           0.23        60
   macro avg       0.19      0.25      0.20        60
weighted avg       0.18      0.23      0.19        60

Confusion Matrix:
 [[ 6  0  9  0]
 [ 5  0 10  3]
 [ 6  0  6  2]
 [ 5  0  6  2]]


/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [ ]:
from sklearn.ensemble import RandomForestClassifier

# Create and train Random Forest model
rf = RandomForestClassifier(random_state=42)
rf.fit(X_train, y_train)

# Predict on test set
y_pred_rf = rf.predict(X_test)

# Evaluate Random Forest
print("Random Forest Accuracy:", accuracy_score(y_test, y_pred_rf))
print("\nClassification Report:\n", classification_report(y_test, y_pred_rf))
print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred_rf))


Random Forest Accuracy: 0.2

Classification Report:
               precision    recall  f1-score   support

   Cancelled       0.00      0.00      0.00        15
   Completed       0.25      0.11      0.15        18
     No-show       0.28      0.36      0.31        14
   Scheduled       0.21      0.38      0.27        13

    accuracy                           0.20        60
   macro avg       0.18      0.21      0.18        60
weighted avg       0.18      0.20      0.18        60

Confusion Matrix:
 [[0 2 5 8]
 [4 2 6 6]
 [2 2 5 5]
 [4 2 2 5]]


In [ ]:
# Create a simplified binary target column
df['status_binary'] = df['status'].replace({
    'Cancelled': 'No-show',
    'No-show': 'No-show',
    'Scheduled': 'Showed-up',
    'Completed': 'Showed-up'
})

# Drop rows with missing binary target if any
df = df.dropna(subset=['status_binary'])

# Define y as the binary target
y = df['status_binary']

# Define X (features) - drop original target and ID columns
X = df.drop([
    'status', 'payment_status', 'appointment_id', 'bill_id', 'treatment_id',
    'patient_id_x', 'patient_id_y', 'doctor_id', 'status_binary'
], axis=1, errors='ignore')

# Add new feature: age from date_of_birth
df['date_of_birth'] = pd.to_datetime(df['date_of_birth'], errors='coerce')
df['age'] = pd.Timestamp.now().year - df['date_of_birth'].dt.year
X['age'] = df['age']

# Drop date_of_birth since age replaces it
X = X.drop('date_of_birth', axis=1, errors='ignore')

# Convert categorical features to numeric
X = pd.get_dummies(X, drop_first=True)

# Split data
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=42
)


In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

log_reg = LogisticRegression(max_iter=10000, class_weight='balanced')
log_reg.fit(X_train, y_train)
y_pred_lr = log_reg.predict(X_test)

print("Logistic Regression Accuracy:", accuracy_score(y_test, y_pred_lr))
print("\nClassification Report:\n", classification_report(y_test, y_pred_lr))
print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred_lr))


Logistic Regression Accuracy: 0.55

Classification Report:
               precision    recall  f1-score   support

     No-show       0.53      0.66      0.58        29
   Showed-up       0.58      0.45      0.51        31

    accuracy                           0.55        60
   macro avg       0.56      0.55      0.55        60
weighted avg       0.56      0.55      0.55        60

Confusion Matrix:
 [[19 10]
 [17 14]]


In [ ]:
from sklearn.ensemble import RandomForestClassifier

rf = RandomForestClassifier(random_state=42, class_weight='balanced')
rf.fit(X_train, y_train)
y_pred_rf = rf.predict(X_test)

print("Random Forest Accuracy:", accuracy_score(y_test, y_pred_rf))
print("\nClassification Report:\n", classification_report(y_test, y_pred_rf))
print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred_rf))


Random Forest Accuracy: 0.5

Classification Report:
               precision    recall  f1-score   support

     No-show       0.48      0.48      0.48        29
   Showed-up       0.52      0.52      0.52        31

    accuracy                           0.50        60
   macro avg       0.50      0.50      0.50        60
weighted avg       0.50      0.50      0.50        60

Confusion Matrix:
 [[14 15]
 [15 16]]


Project Summary
Dataset and Merging:
I used a multi-relational healthcare dataset composed of five tables: patients, appointments, billing, treatments, and doctors. These tables were merged based on common keys such as patient_id, appointment_id, and doctor_id to create a comprehensive dataset suitable for analysis.

Feature Preparation and Target Variable:
I cleaned the data by removing identifier columns and irrelevant features. I also created a binary target variable, status_binary, which simplified the original appointment status into two classes: "No-show" (including Cancelled and No-show) and "Showed-up" (including Scheduled and Completed). I transformed categorical variables into numerical format using one-hot encoding and engineered features such as patient age and appointment period (morning/afternoon).

Models Trained:
Two classification models were trained and evaluated:

Logistic Regression with class weighting to handle class imbalance.

Random Forest Classifier with class weighting.

Evaluation Results:

Logistic Regression achieved approximately 55% accuracy, showing better overall precision and recall compared to Random Forest.

Random Forest achieved approximately 50% accuracy, performing similarly but slightly lower than Logistic Regression.

Both models showed potential but still struggled with some misclassifications, as seen in confusion matrices.

Learnings and Possible Improvements:
The logistic regression model performed slightly better, possibly due to the dataset’s size and feature characteristics. The models could be improved by tuning hyperparameters, adding more features such as days since registration or appointment day of the week, and employing more advanced algorithms. Additionally, techniques like cross-validation and handling data imbalance more rigorously could increase performance.

